In [ ]:
# VYTVORENIE DATASETU overcontact_Bessell_U.pkl

In [1]:
# BLOK 1
# Importovanie kniznic.

import numpy as np
import pandas as pd
import sqlite3
import io

In [2]:
# BLOK 2
# Funkcia pre vyber svetelnej krivky z nespracovanych dat. Zadefinovanie cesty k databaze (subor .db).

def get_curve(txt):
    out = io.BytesIO(txt)
    out.seek(0)
    return np.load(out)

FILEPATH = '../../../data-upjs/overcontact.db'

In [3]:
# BLOK 3 
# Vytvorenie pripojenia na subor. Vypis vsetkych tabuliek v databaze.

conn = sqlite3.connect(FILEPATH)

sql_query = """SELECT name FROM sqlite_master WHERE type='table';"""
cursor = conn.cursor()
cursor.execute(sql_query)
print(cursor.fetchall())

[('parameters',), ('curves',), ('auxiliary',)]


In [4]:
# BLOK 4
# Nacitanie parametrov.

df_parameters = pd.read_sql_query("SELECT * FROM parameters", conn)

In [5]:
# BLOK 5
# Nacitanie svetelnych kriviek.

df_curves = pd.read_sql_query("SELECT * FROM curves", conn)

In [6]:
# BLOK 6
# Vytvorenie tabulky. Jeden riadok = jeden system zakrytovych premennych vo filtri Bessell_U.

df1 = df_curves[["id", "Bessell_U"]]
df2 = df_parameters[["id","primary__t_eff", "secondary__t_eff", "inclination", "mass_ratio", "primary__surface_potential", "secondary__surface_potential"]]
df_merged = pd.merge(df1,df2,on="id")
df_merged["Bessell_U"] = df_merged["Bessell_U"].apply(get_curve)

In [7]:
# BLOK 7
# Uzavretie pripojenia na databazu. Ulozenie dat v podobe tabulky do suboru .pkl.

conn.close()
df_merged.to_pickle("overcontact_Bessell_U.pkl") 